<a href="https://colab.research.google.com/github/cedro3/others2/blob/main/articulated_animation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# セットアップ

In [ ]:
# 1.git-lfsインストール
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!sudo apt-get install git-lfs
!git lfs install

# 2.githubからコードをコピー
!git clone https://github.com/snap-research/articulated-animation.git
%cd articulated-animation

# --- パッチ（2021.9.1追加） ---
import gdown
gdown.download('https://drive.google.com/uc?id=1YHhLDf7QGhVUyAIMRsJBmCm0VDDcQdVV', './pt.zip', quiet=False)
! unzip pt.zip -d checkpoints

# 3.学習済みパラメータをダウンロード
from demo import load_checkpoints
generator, region_predictor, avd_network = load_checkpoints(config_path='config/ted384.yaml',
                                                            checkpoint_path='checkpoints/ted384.pth')

# 4.サンプル画像をダウンロード
! pip install --upgrade gdown
import gdown
gdown.download('https://drive.google.com/uc?id=1ZF6fuBTjfVYKeSpX-0R_nMgYTCjSkxHB', './sample.zip', quiet=False)
! unzip sample.zip -d sup-mat

# 静止画と動画の読み込み

In [ ]:
# --- 静止画と動画の読み込み ---
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

source_image = imageio.imread('sup-mat/001.png')
driving_video = imageio.mimread('sup-mat/driving.mp4')


source_image = resize(source_image, (384, 384))[..., :3]
driving_video = [resize(frame, (384, 384))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
    
HTML(display(source_image, driving_video).to_html5_video())

# 推論・アニメーションの作成

In [ ]:
# --- 推論・アニメーションの作成 ---
from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, 
                             region_predictor, avd_network, animation_mode='avd')

#save resulting video
imageio.mimsave('generated.mp4', [img_as_ubyte(frame) for frame in predictions])

HTML(display(source_image, driving_video, predictions).to_html5_video())